In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import openai
import os

In [6]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("doc2023.pdf"),
    PyPDFLoader("doc2024.pdf"),
    ]
docs = []
for loader in loaders:
    docs.extend(loader.load())

ValueError: File path doc2023.pdf is not a valid file or url

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

#Create a split of the document using the text splitter
splits = text_splitter.split_documents(docs)


In [10]:
os.environ["OPENAI_API_KEY"] = "sk-proj-hegQLFsJdDkZ4Od0H7nzT3BlbkFJ85wscHePDEop2V242Bqc"

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import openai

#openai.api_key = "sk-proj-hegQLFsJdDkZ4Od0H7nzT3BlbkFJ85wscHePDEop2V242Bqc"
embedding = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
persist_directory = 'docs/chroma/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

103


In [15]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
print(vectordb._collection.count())

103


In [16]:
question = "how much more do jhb residents spend than other south african residents?"
docs = vectordb.similarity_search(question,k=3)
len(docs)
print(docs[1])

page_content='Where do the real South African foodies live?\nAlthough Cape Town spends the most on groceries,  \nJohannesburg spends the most on eating out – but Cape Town comes in a close second!\nSA average food spend per active card 2023 (indexed to Johannesburg groceries)\nSource: VisaNet data sample 2023.0.40\n0.91\n1.00\n0.51\n1.09\n0.49\nJohannesburgDurbanCape Town\nGROCERIES EATING OUTFAVOURITE\nday to buy groceries\nSaturday  \n(in-person)Monday   \n(online)\nFAVOURITE\nin-person merchantsA breakdown of the grocery-grab  \n \nThe eating inclinations of Discovery Bank clients\nOnline grocery spending is now spread more evenly \nacross the week as consumers embrace same-day \ndelivery services to buy groceries when they need them.\nDiscovery Bank weekly distribution of total grocery transactions by day (2023)\nONLINE IN-PERSONThursday Friday Saturday Sunday Wednesday16%\n16% 15%15%\n15%17%\n13%13% 13% 13%13%\nTuesday Monday14%\n14%14%\nSource: Exclusively Discovery Bank.FAVOURIT

In [23]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-4"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [18]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [22]:
question = "how much has grocery spend increased by?"
result = qa_chain({"query": question})
result["result"]

'In the past year, the average grocery spending in South Africa rose by 8%. For the mass market clients, grocery spend has increased by approximately 50% since 2019.'

In [24]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end.
 If you don't know the answer, just say that you don't know, don't try to make up an answer.
 Use three sentences maximum. Keep the answer as concise as possible.
 Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [29]:
question = "how much has grocery spend increased by?"
result = qa_chain({"query": question})
# Check the result of the query
result["result"]
# Check the source document from where we
#result["source_documents"][0]

'In the past year, the average grocery spending in South Africa increased by 8%. However, for mass market clients, grocery spending has increased by approximately 50% since 2019. Thanks for asking!'